In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [ ]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
WHERE Year=2024 AND GP="Australian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

In [ ]:
# GP=df_bq[((df_bq["GP"]=="Australian Grand Prix") & (df_bq["Year"]==2024))]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
GP.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
985,0 days 01:00:05.328000,ALB,23,0 days 00:01:35.752000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.858000,0 days 00:00:38.140000,NaT,0 days 00:59:27.287000,0 days 01:00:05.463000,265.0,310.0,292.0,238.0,False,MEDIUM,1.0,True,Williams,0 days 00:58:29.232000,NaT,1,14.0,False,,False,False,2024,Australian Grand Prix
99,0 days 01:01:29.864000,ALB,23,0 days 00:01:24.536000,2,1.0,NaT,NaT,0 days 00:00:30.021000,0 days 00:00:18.135000,0 days 00:00:36.380000,0 days 01:00:35.294000,0 days 01:00:53.429000,0 days 01:01:29.809000,273.0,321.0,306.0,279.0,True,MEDIUM,2.0,True,Williams,0 days 01:00:05.328000,NaT,1,14.0,False,,False,True,2024,Australian Grand Prix
100,0 days 01:02:53.470000,ALB,23,0 days 00:01:23.606000,3,1.0,NaT,NaT,0 days 00:00:29.307000,0 days 00:00:18.047000,0 days 00:00:36.252000,0 days 01:01:59.116000,0 days 01:02:17.163000,0 days 01:02:53.415000,269.0,319.0,305.0,312.0,True,MEDIUM,3.0,True,Williams,0 days 01:01:29.864000,NaT,1,14.0,False,,False,True,2024,Australian Grand Prix
101,0 days 01:04:16.908000,ALB,23,0 days 00:01:23.438000,4,1.0,NaT,NaT,0 days 00:00:29.061000,0 days 00:00:18.076000,0 days 00:00:36.301000,0 days 01:03:22.476000,0 days 01:03:40.552000,0 days 01:04:16.853000,272.0,322.0,304.0,NaN,True,MEDIUM,4.0,True,Williams,0 days 01:02:53.470000,NaT,1,13.0,False,,False,True,2024,Australian Grand Prix
531,0 days 01:05:40.824000,ALB,23,0 days 00:01:23.916000,5,1.0,NaT,NaT,0 days 00:00:29.328000,0 days 00:00:18.079000,0 days 00:00:36.509000,0 days 01:04:46.181000,0 days 01:05:04.260000,0 days 01:05:40.769000,275.0,325.0,306.0,308.0,False,MEDIUM,5.0,True,Williams,0 days 01:04:16.908000,NaT,1,13.0,False,,False,True,2024,Australian Grand Prix


In [ ]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [ ]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [ ]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)

# GP_Rk["RK"]=GP_Rk["RK"].astype(int)
GP_FASTEST_LAPS=GP_Rk[GP_Rk["LAP_RK"]==1.0].sort_values("LapNumber")
GP_FASTEST_LAPS=GP_FASTEST_LAPS[["Time","Driver","Team","LapTime","LapNumber","Stint","PitOutTime","PitInTime","Sector1Time","Sector2Time","Sector3Time","SpeedI1","SpeedI2","TyreLife","TrackStatus","Position","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_FASTEST_LAPS.head()

,Time,Driver,Team,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,TyreLife,TrackStatus,Position,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
977,0 days 00:59:57.034000,VER,Red Bull Racing,00:01:27.458000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.462000,0 days 00:00:35.518000,278.0,285.0,1.0,1,1.0,2024,Australian Grand Prix,1.0,NaN,5.0,1.0
176,0 days 01:01:20.551000,SAI,Ferrari,00:01:22.600000,2,1.0,NaT,NaT,0 days 00:00:29.006000,0 days 00:00:18.026000,0 days 00:00:35.568000,277.0,320.0,2.0,1,1.0,2024,Australian Grand Prix,1.0,3.0,5.0,1.0
160,0 days 01:02:45.199000,NOR,McLaren,00:01:22.656000,3,1.0,NaT,NaT,0 days 00:00:28.750000,0 days 00:00:18.204000,0 days 00:00:35.702000,271.0,299.0,3.0,1,3.0,2024,Australian Grand Prix,1.0,2.0,15.0,2.0
177,0 days 01:04:05.990000,SAI,Ferrari,00:01:22.433000,4,1.0,NaT,NaT,0 days 00:00:28.643000,0 days 00:00:18.497000,0 days 00:00:35.293000,270.0,291.0,4.0,1,1.0,2024,Australian Grand Prix,1.0,1.0,18.0,1.0
935,0 days 01:05:31.688000,PIA,McLaren,00:01:22.594000,5,1.0,NaT,NaT,0 days 00:00:28.691000,0 days 00:00:17.918000,0 days 00:00:35.985000,NaN,324.0,5.0,1,4.0,2024,Australian Grand Prix,1.0,1.0,2.0,6.0


In [21]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP1=GP[GP["Driver"]=="SAI"]
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
985,ALB,1,NaN,18.858,38.140,00:01:35.752000,MEDIUM,2024,Australian Grand Prix,14.0,NaN,18.0,15.0
980,ALO,1,NaN,18.774,37.212,00:01:34.503000,HARD,2024,Australian Grand Prix,11.0,NaN,15.0,11.0
994,BOT,1,NaN,18.782,37.741,00:01:34.763000,MEDIUM,2024,Australian Grand Prix,12.0,NaN,16.0,13.0
978,GAS,1,NaN,18.854,38.564,00:01:37.304000,MEDIUM,2024,Australian Grand Prix,17.0,NaN,17.0,18.0
991,HAM,1,NaN,18.674,36.983,00:01:33.688000,SOFT,2024,Australian Grand Prix,10.0,NaN,11.0,8.0


In [19]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig = go.Figure(go.Table
 (header={"values": GP_Sectors.columns},
  cells=dict(
    values=GP_Sectors.T.values,
    # fill_color=[color: 'darkorange' if GP_Sectors["S1_RK"]==1 else color: 'darkblue' for v in GP_Sectors.T.values],
    align='center', font=dict(color='black', size=11)
    )
    )
 )

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": GP_Sectors.T.values
                                if c == "All"
                                else GP_Sectors.loc[GP_Sectors["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + GP_Sectors["LapNumber"].unique().tolist()
            ]
        }
    ]
)

fig.show()

In [17]:
import plotly.express as px

x = ['Team A', 'Team B', 'Team C']
y = ['Game One', 'Game Two', 'Game Three']

z = [[.1, .3, .5],
     [1.0, .8, .6],
     [.6, .4, .2]]

z_text = [['Win', 'Lose', 'Win'],
          ['Lose', 'Lose', 'Win'],
          ['Win', 'Win', 'Lose']]

fig = px.imshow(z, x=x, y=y, color_continuous_scale='Viridis', aspect="auto")
fig.update_traces(text=z_text, texttemplate="%{text}")
fig.update_xaxes(side="top")
fig.show()

In [16]:
import plotly.express as px
data=[[1, 25, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, 5, 20]]
fig = px.imshow(data,
                labels=dict(x="Day of Week", y="Time of Day", color="Productivity"),
                x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                y=['Morning', 'Afternoon', 'Evening']
               )
fig.update_xaxes(side="top")
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')
a = np.random.randint(low=0, high=9, size=10)
b = np.random.randint(low=0, high=9, size=10)
c = np.random.randint(low=0, high=9, size=10)

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Column A</b>', '<b>Column B</b>', '<b>Column C</b>'],
    line_color='white', fill_color='white',
    align='center',font=dict(color='black', size=12)
  ),
  cells=dict(
    values=[a, b, c],
    line_color=[np.array(colors)[a],np.array(colors)[b], np.array(colors)[c]],
    fill_color=[np.array(colors)[a],np.array(colors)[b], np.array(colors)[c]],
    align='center', font=dict(color='white', size=11)
    ))
])

fig.show()

In [ ]:
#Multiple plotly filters code
import plotly.graph_objects as go

df = pd.DataFrame(
    {
        "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
        "A_item": [2, 8, 0],
        "B_item": [1, 7, 10],
        "C_item": [9, 2, 9],
        "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
    }
)

fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
fig.update_layout(
    updatemenus=[
        {
            "y": 1 - (i / 5),
            "buttons": [
                {
                    "label": c,
                    "method": "restyle",
                    "args": [
                        {
                            "cells": {
                                "values": df.T.values
                                if c == "All"
                                else df.loc[df[menu].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + df[menu].unique().tolist()
            ],
        }
        for i, menu in enumerate(["Channel_type", "Date"])
    ]
)

In [ ]:
#Single Plotly filter code
import plotly.graph_objects as go

df = pd.DataFrame(
    {
        "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
        "A_item": [2, 8, 0],
        "B_item": [1, 7, 10],
        "C_item": [9, 2, 9],
        "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
    }
)

fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": df.T.values
                                if c == "All"
                                else df.loc[df["Channel_type"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + df["Channel_type"].unique().tolist()
            ]
        }
    ]
)